This code visualize HSAF H35 (Effective Snow Cover) between given coordinates

In [1]:
import warnings
warnings.simplefilter("ignore")

import gc
from operator import length_hint
import numpy as np
from datetime import datetime,timedelta
from glob import glob
import gzip

import pygrib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.basemap import Basemap

In [2]:
input_dir  = "../input/"
output_dir = "../output/"
fziplist_     = glob(input_dir+"h35*gz")
latcorners = [33,45]
loncorners = [23,48]
total_files=length_hint(fziplist_)

In [3]:
def gunzip(infilename):
    # this function unzip a .gz file
    input = gzip.GzipFile(infilename, 'rb')
    s = input.read()
    input.close()
    output = open(infilename[0:-3], 'wb')
    output.write(s)
    output.close()
    return infilename[0:-3]

In [4]:
flist_=[]

for zips in fziplist_:
    gunzip(zips)
    flist_.append(zips[0:-3])

In [5]:
rgb = np.load("../auxilary/H35_cmap.npz")
newcmp = colors.ListedColormap(rgb["rgb"], "")

In [6]:
file_=flist_[0]
date_=file_.split("/")[-1].split(".")[0].split("_")[1]
grbs = pygrib.open(file_)
grb = grbs.select()[0]
del grbs
data = grb.values
lons = np.linspace(float(grb['longitudeOfFirstGridPointInDegrees'])-360 ,\
                   float(grb['longitudeOfLastGridPointInDegrees']), int(grb['Ni']))
lats = np.linspace(float(grb['latitudeOfFirstGridPointInDegrees']), \
                   float(grb['latitudeOfLastGridPointInDegrees']), int(grb['Nj']))
del grb
lat0=np.where(lats<=latcorners[0])[0][0]
lat1=np.where(lats<=latcorners[1])[0][0]
lon0=np.where(lons>=loncorners[0])[0][0]
lon1=np.where(lons>=loncorners[1])[0][0]
trk_esc=data[lat1:lat0,lon0:lon1]
del data
trk_lons=lons[lon0:lon1]
trk_lats=lats[lat1:lat0]
del lons,lats
trk_lon,trk_lat=np.meshgrid(trk_lons,trk_lats)
del trk_lons,trk_lats
m = Basemap(projection='merc', resolution='h', area_thresh=1000.,\
            llcrnrlat=latcorners[0],urcrnrlat=latcorners[1],\
            llcrnrlon=loncorners[0],urcrnrlon=loncorners[1])
m.drawcountries()
m.drawcoastlines()
m1=m.pcolor(trk_lon,trk_lat,trk_esc,latlon=True,cmap=newcmp)
plt.axis('off')
plt.savefig(output_dir+"Turk_H35_"+date_+".png", transparent=True,\
          dpi=200, bbox_inches='tight', pad_inches = 0)
plt.close()
gc.collect()

0

In [7]:
from PIL import Image

#read the image
im = Image.open(output_dir+"Turk_H35_"+date_+".png")

#show image
im.show()